In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from main_functions import *
from DGD_DP import DGD_DP as dgd_dp
import pickle
import os
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

class instance:
    def __init__(self,X,centers_kernel,Y,kernel_center_of_agent,data_point_of_agent,communication_matrix,sigma) -> None:
        self.centers_kernel=centers_kernel
        self.X = X
        self.Y=Y
        self.kernel_center_of_agent = kernel_center_of_agent
        self.data_point_of_agent = data_point_of_agent
        self.communication_matrix = communication_matrix
        self.sigma = sigma
        number_of_kernels = np.shape(centers_kernel)[0]
        number_of_datas = np.shape(X)[0]
        self.number_of_datas = number_of_datas
        self.number_of_kernels = number_of_kernels
        self.Kmm_matrix = np.array([[kernel(centers_kernel[k],centers_kernel[l]) for k in range(number_of_kernels)]for l in range(number_of_kernels)])
        self.Knm_matrix = np.array([[kernel(centers_kernel[k],X[l]) for k in range(number_of_kernels)]for l in range(self.number_of_datas)])
        number_of_agents,kernels_per_agent = np.shape(kernel_center_of_agent)
        _,data_per_agent = np.shape(data_point_of_agent)
        self.number_of_agents = number_of_agents
        self.kernels_per_agent = kernels_per_agent
        self.data_per_agent = data_per_agent
        matrixA = np.identity(number_of_agents*number_of_kernels)
        for index in range(number_of_kernels):
                matrixA[(number_of_agents-1)*number_of_kernels+index,(number_of_agents-1)*number_of_kernels+index]=0
        for index in range((number_of_agents-1)*number_of_kernels):
                matrixA[index,index+number_of_kernels]=-1
        self.matrixA = matrixA
        self.true_solution = np.zeros(number_of_agents)

    def objective(self,curent_solution):
        objecti = np.dot(curent_solution,np.dot(self.Kmm_matrix,curent_solution))/2
        for agent_index in range(self.number_of_agents):
            for agent_index_2 in range(self.data_per_agent):
                matrix_Kim = np.array([kernel(self.X[self.data_point_of_agent[agent_index,agent_index_2]],self.centers_kernel[j]) for j in range(self.number_of_kernels)])
                objecti+=(1/(2*(self.sigma**2)))*(self.Y[self.data_point_of_agent[agent_index,agent_index_2]]-np.dot(matrix_Kim,curent_solution))**2
        return(objecti)
    
    def gradient(self,agent_index,point):
        sum  = np.dot(self.Kmm_matrix,point)/5    
        for agent_index_2 in range(self.data_per_agent):
            matrix_Kim = np.array([kernel(self.X[self.data_point_of_agent[agent_index,agent_index_2]],self.centers_kernel[j]) for j in range(self.number_of_kernels)])
            sum -=matrix_Kim*(self.Y[self.data_point_of_agent[agent_index,agent_index_2]]-np.dot(matrix_Kim,point))/(self.sigma**2)
        return sum
    
    def complete_gradient(self,point):
        product = np.dot(self.Knm_matrix.T,self.Knm_matrix) 
        grad = np.dot(self.Kmm_matrix+(1/self.sigma**2)*product,point)
        grad -= (1/self.sigma**2)*np.dot(self.Y.T,self.Knm_matrix)
        return(grad)
    
    def partial_gradient(self,agent_index,point,agent_index_2):
        sum  = np.dot(self.Kmm_matrix,point)/5    
        matrix_Kim = np.array([kernel(self.X[self.data_point_of_agent[agent_index,agent_index_2]],self.centers_kernel[j]) for j in range(self.number_of_kernels)])
        sum -=matrix_Kim*(self.Y[self.data_point_of_agent[agent_index,agent_index_2]]-np.dot(matrix_Kim,point))/(self.sigma**2)
        return sum

In [3]:
with open(os.path.join("data","first_database.pkl"), "rb") as f:
    x,y = pickle.load(f)
n= 100
m = 10
#mélange et séléction des données
X = x[:n]
Y = y[:n]
stack = np.stack([X,Y]).T
np.random.shuffle(stack)
x_shuffle = stack.T[0]
y_shuffle = stack.T[1]
centers_kernel = np.array([x_shuffle[10*i] for i in range(10)])
kernel_center_of_agent = np.array([[0,1],[2,3],[4,5],[6,7],[8,9]])
data_point_of_agent = np.array([np.arange(20*i,20*i+20) for i in range(5)])

In [4]:
#matrices d'adjacence pour dual decomposition et ADMM
A1 = np.ones((5,5)) #fully connected
A2 = np.array([[1,0,1,1,1],[0,1,1,0,1],[1,1,1,1,1],[1,0,1,1,0],[1,1,1,0,1]]) #small world
A3 = np.array([[1,1,0,0,1],[1,1,1,0,0],[0,1,1,1,0],[0,0,1,1,1],[1,0,0,1,1]]) #cycle
A4 = np.array([[1,1,0,0,1],[1,1,1,0,0],[0,1,1,1,0],[0,0,1,1,1],[0,0,0,1,1]]) #line
A5= np.array([[1,1,1,1,0],[1,1,1,1,0],[1,1,1,1,0],[1,1,1,1,0],[0,0,0,0,1]]) #non connexe

In [5]:
instance_pb = instance(x_shuffle,centers_kernel,y_shuffle,kernel_center_of_agent,data_point_of_agent,A2,0.5)

In [6]:
solver_non_cooperative = gradient_descent(instance_pb,0.0001,2000,np.ones(10))
true_solution = solver_non_cooperative.solve(verbose=False)
instance_pb.true_solution =  true_solution
print(solver_non_cooperative.curent_solution)

[-0.27147904  0.82040863  0.0324456  -0.22228674 -1.17575479  0.98455146
 -0.36616175 -0.71972321  0.77912622  1.05490557]


In [29]:
solver_DGD = dgd_dp(instance_pb,0.0001, 20,np.ones(10))
solver_DGD.solve(laplacian_scale=1e-2,gamma_init=0.1,verbose=False,package_loss = False, probability_package_loss = 0, decay=0.3)
print(solver_DGD.current_solution)

[[ 9.80972178e+12 -1.17986244e+13  7.04713784e+12  2.13387139e+12
   3.84276687e+13  1.65445953e+13 -3.71416857e+13 -1.63076199e+13
  -2.05675662e+13 -2.75329985e+13]
 [ 1.31741039e+13 -1.42731170e+13  1.02263530e+13  1.67697563e+12
   3.44661201e+13  2.26248538e+13 -3.56513891e+13 -2.03310134e+13
  -1.45650852e+13 -3.81932042e+13]
 [-6.22326533e+13  6.41083422e+13 -4.98700727e+13 -5.30716195e+12
  -1.26666900e+14 -1.07681928e+14  1.38394830e+14  9.27107961e+13
   4.13063266e+13  1.83041000e+14]
 [ 1.79350076e+13 -1.77810089e+13  1.47112166e+13  9.98772417e+11
   2.88637380e+13  3.12195775e+13 -3.35480478e+13 -2.60236622e+13
  -6.08293652e+12 -5.33066839e+13]
 [ 2.13050372e+13 -2.02624447e+13  1.78770936e+13  4.89773372e+11
   2.48993142e+13  3.72874153e+13 -3.20626441e+13 -3.00572573e+13
  -9.66114520e+10 -6.40145201e+13]]


In [27]:
solver_DGD = DGD(instance_pb,0.001,1000,np.ones(10))
solver_DGD.solve(verbose=False,package_loss = False, probability_package_loss = 0)
print(solver_DGD.curent_solution)

[[-0.26335096  0.80971154  0.03540361 -0.21253817 -1.15235523  0.97566671
  -0.35643218 -0.70249016  0.77345697  1.04006446]
 [-0.28154899  0.7906144   0.01691984 -0.230948   -1.16960515  0.95642908
  -0.37453668 -0.7203689   0.75430184  1.02082927]
 [-0.26597256  0.80707606  0.03280367 -0.2185789  -1.15554381  0.96803688
  -0.35906747 -0.70767437  0.76606097  1.03740463]
 [-0.26060644  0.81207753  0.03799405 -0.20508924 -1.14836513  0.98446856
  -0.35362777 -0.69592278  0.78208844  1.0423959 ]
 [-0.26382831  0.80684637  0.03422995 -0.21145497 -1.15041305  0.97464237
  -0.35668623 -0.7006237   0.77280174  1.03672332]]
